# ***INSTALLING PACKAGES***

In [4]:
# !conda install -c conda-forge python-telegram-bot
# # # !conda install preprocessor
# # # !conda install emoji
# # # !conda install spacy
# # # !conda install transformers # > 2.2.0
# # # !conda install neuralcoref
# # # !python -m spacy download en_core_web_md
# # # !conda install bert-extractive-summarizer
# !pip install bert-extractive-summarizer
# !pip install python-telegram-bot
# !pip install transformers==2.4.1
# !curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
!pip install torch torchvision torchaudio

ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


# ***IMPORTING LIBRARIES***

In [3]:
from telegram.ext import *
from datetime import datetime
import time
from summarizer import Summarizer
import pytz
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score
import pickle
import tweepy as tw
import pandas as pd
import csv
import re 
import string
import preprocessor as p
import json
import os
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

ModuleNotFoundError: No module named 'torch'

# ***MOVIE AND YEAR STRING CREATION***

In [ ]:
def tweetMovie(movieName, Date):
  
  movieName = movieName.replace(" ","")
  S1 = "#" + movieName
  S2 = "#" + movieName + "review"
  S3 = "#" + movieName + Date
  Date = Date + "-01-01"
  S = S1 + " OR " + S2 + " OR " + S3

  return S, Date

# ***IMDB DATASET EXTRACTION AND CLEANING***

In [ ]:
from google.colab import drive
drive.mount('/content/Drive/', force_remount = True)
imdbData = pd.read_csv('/content/Drive/MyDrive/Term 2/CBD 3335/Final Project/IMDB Dataset.csv', header = 0)

print(imdbData.shape)
imdbData.head()

In [ ]:
imdbData.sentiment.replace('positive',1,inplace=True)
imdbData.sentiment.replace('negative',0,inplace=True)
imdbData.rename(columns={'review': 'Review', 'sentiment': 'Sentiment'}, inplace=True)
imdbData.head(10)

In [ ]:
def Clean(Text):
    
    Cleaned = re.compile(r'<.*?>')
    return re.sub(Cleaned, '', Text)

imdbData.Review = imdbData.Review.apply(Clean)
imdbData.Review[0]

In [ ]:
def isSpecial(Text):
    
    Rem = ''
    for i in Text:
        
        if i.isalnum():
            Rem = Rem + i
        else:
            Rem = Rem + ' '
    
    return Rem

imdbData.Review = imdbData.Review.apply(isSpecial)
imdbData.Review[0]

In [ ]:
def toLower(Text):
    
    return Text.lower()

imdbData.Review = imdbData.Review.apply(toLower)
imdbData.Review[0]

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

def remStopwords(Text):
    
    stopWords = set(stopwords.words('english'))
    Words = word_tokenize(Text)
    
    return [W for W in Words if W not in stopWords]

imdbData.Review = imdbData.Review.apply(remStopwords)
imdbData.Review[0]

In [ ]:
def stemText(Text):
    
    SS = SnowballStemmer('english')
    
    return " ".join([SS.stem(W) for W in Text])

imdbData.Review = imdbData.Review.apply(stemText)
imdbData.Review[0]

# ***TRAIN TEST SPLIT***

In [ ]:
X = np.array(imdbData.iloc[:,0].values)
Y = np.array(imdbData.Sentiment.values)
CV = CountVectorizer(max_features = 1000)
X = CV.fit_transform(imdbData.Review).toarray()

print("X Shape = ", X.shape)
print("y Shape = ", Y.shape)

In [ ]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=9)

print("Train Shapes : X = {}, Y = {}".format(trainX.shape, trainY.shape))
print("Test Shapes : X = {}, Y = {}".format(testX.shape, testY.shape))

# ***MODEL SELECTION***

In [ ]:
GNB, MNB, BNB = GaussianNB(), MultinomialNB(alpha=1.0,fit_prior=True), BernoulliNB(alpha=1.0,fit_prior=True)
GNB.fit(trainX, trainY)
MNB.fit(trainX, trainY)
BNB.fit(trainX, trainY)

In [ ]:
YPG = GNB.predict(testX)
YPM = MNB.predict(testX)
YPB = BNB.predict(testX)

print("Gaussian = ", accuracy_score(testY, YPG))
print("Multinomial = ", accuracy_score(testY, YPM))
print("Bernoulli = ", accuracy_score(testY, YPB))

# ***TWITTER API VALUES***

In [ ]:
accessKey = '1076801107-YnSnzPG8QVLCHTpAIzH0B0iN5y4Ktg3ooWBtC32'
accessSecret = 'tu6pZ59yae4PbBWl7VamRjUgLzaV3Dl1hWpjS96tvCCjk'
consumerKey = 'yUv9lg35fpu56VJPPgxx54KQI'
consumerSecret ='hGgAALXQrIiB86gSmFxHqj3Qm3RRHEmPHh1sq5UW702tR32k1D'

In [ ]:
Auth = tw.OAuthHandler(consumerKey, consumerSecret)
Auth.set_access_token(accessKey, accessSecret)
API = tw.API(Auth, wait_on_rate_limit=True)

# ***TELEGRAM BOT***

In [ ]:
MOVIE, YEAR = range(0,2)

def Start(Update, CallbackContext):

    Update.message.reply_text('Hello! This Is Critique - Automated Movie Review System!\n\nType The Name Of The Movie:')

    return MOVIE

def Movie(Update, CallbackContext):
    
    global movieName 
    movieName = Update.message.text
    Update.message.reply_text('\nType The Year Of The Movie:')
    
    return YEAR

def Year(Update, CallbackContext):

    movieYear = Update.message.text
    searchWords, dateSince = tweetMovie(movieName, movieYear)
    newSearch = searchWords + " -filter:links"
    Update.message.reply_text("\n\nBot At Work, Please Wait...\n\n")

    #TWITTER DATA COLLECTION
    Tweets = tw.Cursor(API.search, 
                           q=newSearch,
                           lang="en",
                           since=dateSince).items(5000)

    usersLocs = [[Tweet.user.screen_name, Tweet.user.location, Tweet.text] for Tweet in Tweets]

    tweetText = pd.DataFrame(data=usersLocs, columns=['User', 'Location', 'Tweet'])

    tweetList = list(tweetText.Tweet)

    cleanTweets = []

    #TWITTER DATA CLEANING
    for Tweet in tweetList:

      Tweet = re.sub("@[A-Za-z0-9]+","", Tweet)
      Tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", Tweet)
      Tweet = " ".join(Tweet.split())
      Tweet = ''.join(c for c in Tweet if c not in emoji.UNICODE_EMOJI['en'])
      Tweet = Tweet.replace("#", "").replace("_", " ")
      Tweet = " ".join(w for w in nltk.wordpunct_tokenize(Tweet) \
          if w.lower() in words or not w.isalpha())
      cleanTweets.append(Tweet)
    
    cleanTweets2=[]

    for Tweet in cleanTweets:
      if(len(Tweet) > 5):
        cleanTweets2.append(Tweet)
    
    #SUMMARIZATION
    movieSummary= " ".join(Tweet for Tweet in cleanTweets2)

    Model = Summarizer()
    Summary = Model(movieSummary)
    
    pickle.dump(BNB,open('Model.pkl','wb'))
    Rev = Summary
    F1 = Clean(Rev)
    F2 = isSpecial(F1)
    F3 = toLower(F2)
    F4 = remStopwords(F3)
    F5 = stemText(F4)

    Bow, Words = [], word_tokenize(F5)
    
    for Word in Words:
        Bow.append(Words.count(Word))
    
    wordDict = CV.vocabulary_
    pickle.dump(wordDict,open('Bow.pkl','wb'))

    Inp = []
    for i in wordDict:
        Inp.append(F5.count(i[0]))
    
    #CLASSIFICATION USING BNB
    yPred = BNB.predict(np.array(Inp).reshape(1,1000))

    #OUTPUTING THE RESULT USING TELEGRAM BOT
    print("Summarized Review :\n", Summary)
    Update.message.reply_text("Summarized Review :\n ")
    Update.message.reply_text(Summary)

    if (yPred==1):
      print("The Movie: ",movieName ," Has Positive Sentiment Reviews")
      Update.message.reply_text("The Movie: " + movieName + " Has Positive Sentiment Reviews.")
    else:
      print("The Movie: ",movieName ," Has Negative Sentiment Reviews")
      Update.message.reply_text("The Movie: " + movieName + " Has Negative Sentiment Reviews.")
    

    Update.message.reply_text("\n\nPress /start To Start A New Review Process Or Goodbye, Fow Now <3.")


    return ConversationHandler.END

def Bye(Update, CallbackContext):

    Update.message.reply_text('Bye! I hope we can talk again some day.', reply_markup=ReplyKeyboardRemove())

    return ConversationHandler.END


def main():

    print("Bot Has Started.")
  
    updaterVar = Updater("2102220766:AAEbLz-RNQjm9SDmilcZhExDgglEnSS-Xe0", use_context=True)
    Dispatcher = updaterVar.dispatcher

    convHandler = ConversationHandler(
        
                                        entry_points=[CommandHandler('start', Start)],
                                        states={
                                            
                                                  MOVIE: [MessageHandler(Filters.text, Movie)],
                                                  YEAR:  [MessageHandler(Filters.text &  ~Filters.command, Year)]
                                              
                                               },

                                        fallbacks=[CommandHandler('Bye', Bye)],
                                    )

    Dispatcher.add_handler(convHandler)
    
    Dispatcher.add_handler(CommandHandler("Start", Start))

    updaterVar.start_polling()
    updaterVar.idle()

main()
print("Bot Has Stopped.")